# Soft-VC demo

In [3]:
import torch
import torchaudio
import IPython.display as display

from hubert import HubertSoft
from acoustic import AcousticModel
import audio as A

device = 'cuda' if torch.cuda.is_available() else 'cpu'

c:\Miniconda3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
hubert     = torch.hub.load("bshall/hubert:main", "hubert_soft"         ).to(device)
hubert_pit = HubertSoft()                                                .to(device)
hubert_dyn = HubertSoft()                                                .to(device)
acoustic   = AcousticModel()                                             .to(device)
hifigan    = torch.hub.load("bshall/hifigan:main", "hifigan_hubert_soft").to(device)

Using cache found in C:\Users\jiangsongru/.cache\torch\hub\bshall_hubert_main
Using cache found in C:\Users\jiangsongru/.cache\torch\hub\bshall_hifigan_main


Removing weight norm...


In [5]:
dataset = 'databaker'

ckpt = torch.load(f'out/{dataset}/hubert-pit/model-best.pt', map_location=device)
hubert_pit.load_state_dict(ckpt["hubert"]) ; del ckpt
ckpt = torch.load(f'out/{dataset}/hubert-dyn/model-best.pt', map_location=device)
hubert_dyn.load_state_dict(ckpt["hubert"]) ; del ckpt

In [6]:
## speech
#vbank = 'ljspeech'
#vbank = 'databaker'
## vocal
#vbank = 'sou'
#vbank = 'len'
#vbank = 'lemi'
vbank = 'hana'
#vbank = 'ema'
#vbank = 'urushi'
#vbank = 'lansi'
## instrumental
#vbank = 'piano'

#steps = 21000
steps = 'best'

ckpt = torch.load(f'out/{vbank}/acoustic/model-{steps}.pt', map_location=device)
print(ckpt['step'])
acoustic.load_state_dict(ckpt["acoustic-model"]); del ckpt

12000


In [9]:
## speech
#wav_fp = 'test/LJ001-0001.wav'
#wav_fp = 'test/000001.wav'
#wav_fp = 'test/BAC009S0724W0121.wav'
#wav_fp = 'test/SSB00050353.wav'
#wav_fp = 'test/common_voice_ab_19904194.wav'
#wav_fp = 'test/000df995-51ab-4a1d-852c-c288e0300bf7.wav'
## vocal (recllst)
wav_fp = 'test/D4_しゅしゅししょしゃしょし.wav'
#wav_fp = 'test/_べべぶぼべぼぼ.wav'
#wav_fp = 'test/_きゅんきゅきゅきぇきゅきょきゅ.wav'
#wav_fp = 'test/kan_kan_kan.wav'
## vocal (song)
#wav_fp = 'test/[Hana] FLOWER_1_歌声-トラック_tune-8.wav'
#wav_fp = 'test/[Ema] さがり花_1_歌声-トラック-6.wav'
#wav_fp = 'test/[Len] テロル_vocal-4.wav'
## instrumental
#wav_fp = 'test/Ⅱ 小快板-14.wav'
#wav_fp = 'test/十二音小夜曲-10.wav'

wav = A.load_wav(wav_fp)
wav = A.trim_silence(wav)
wav = A.align_wav(wav)
wav = torch.from_numpy(wav).unsqueeze(0).unsqueeze(0)
wav = wav.to(device)

print('original:')
display.Audio(wav.squeeze().cpu(), rate=16000)

original:


In [10]:
with torch.inference_mode():
  # Extract speech units
  units    = hubert    .units(wav)
  embd_pit = hubert_pit.units(wav)
  embd_dyn = hubert_dyn.units(wav)
  units_pitdyn = torch.cat([units, embd_pit, embd_dyn], axis=-1)
  # Generate target spectrogram
  mel = acoustic.generate(units_pitdyn).transpose(1, 2)
  # Generate audio waveform
  target = hifigan(mel)

print('converted:')
display.Audio(target.squeeze().cpu(), rate=16000)

converted:
